# Texas Hold'em

In this project, I will design a user interactive game of Texas Hold'em. There will be many instances of hands as the deal proceeds around the table. There will be rounds of betting held by each player at the table. If the player at the table is not human, the player will make random decisions about betting. 

Because this will involve interaction between many different classes, some inherited, it will broadly cover the topics from the course thus far.

In [1]:
import itertools as it
import random as rand

In [2]:
class Player():
    def __init__(self,name):
        '''Initialize traits of any player, including:
        
        Hand        - List of length 2.
        Balance     - Dollars stored as float
        Name        - Stored as string
        Hand Status - String in list ["call","check","raise","fold"]
        Player Bet    - The last bet the Player made.
        Dealer      - Whether you're the dealer or not
        Best Hand   - The 5 card hand you have
        Best Hand Score - The score associated with that hand
        Raiser      - Whether you are the last player to raise
        '''
        self.player_hand = []
        self.balance = 1000
        self.name = name
        self.hand_status = None
        self.player_bet = 0
        self.dealer = 0
        self.best_hand = []
        self.best_hand_score = []
        self.raiser = 0
    
    def players_best_hand(self, full_hand):
        '''When called, instantiates all possible hands of existing set and selects the best one. Assigns it to self.best_hand'''
        if len(full_hand) < 5:
            self.best_hand = self.player_hand
        else:
            all_possible_hands = [Hand(x) for x in list(it.combinations(full_hand,5))]
            all_possible_hands = sorted(all_possible_hands)[::-1]
            self.best_hand = all_possible_hands[0]
            self.best_hand_score = max([x.hand_score for x in all_possible_hands])
        
    def __hash__(self):
        return hash(self.name)
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
    
class Bot(Player):
    def make_decision(self, table_bet, table_hand):
        """Randomly makes a decision about Hand Status within the rules of poker.
        
        If current_bet is an integer, tell the player the current bet is current_bet.
        Limits options to ["call","raise","fold"]
        If current_bet is a string, it must be check, so tell him that everyone else checked.
        Limits options to ["call","bet","fold"]
        
        Returns self.hand_status - String in list ["call","check","raise","fold"]
        """
        
        #when the table_bet is None, everybody else has checked thus far
        #so choose from either check or call
        if table_bet == 0:
            choice_set = ["check","raise"]
            
        #otherwise, somebody has bet before you
        #if that's the case, choose from check, bet, or fold
        else:
            if table_bet == self.balance:
                choice_set = ["call","fold"]
            elif table_bet > self.balance:
                choice_set = ["fold"]
            else:
                choice_set = ["call","raise","fold"]
        
        #make your random choice
        self.hand_status = choice_set[rand.choice(list(range(len(choice_set))))]
        #if you select raise, find a number between the existing bet and your balance to bet
        #minimum raise of 10 and must be a multiple of 10
        if self.hand_status == 'raise':
            self.player_bet = int(round(rand.choice(list(range(1,(int(self.balance)-int(table_bet)))))/5))
            self.balance -= self.player_bet
            print("{0} {1}s {2}.".format(self.name,self.hand_status,self.player_bet))
            print("The last bet on the table is {0}.".format(table_bet + self.player_bet))
        elif self.hand_status == 'call':
            self.player_bet = table_bet
            self.balance -= self.player_bet
            print("{0} {1}s.".format(self.name,self.hand_status))
        elif self.hand_status == 'check':
            self.player_bet = 0
            print("{0} {1}s.".format(self.name,self.hand_status))
        elif self.hand_status == 'fold':
            print("{0} {1}s.".format(self.name,self.hand_status))
        return self.player_bet
    
class Human(Player):
    def make_decision(self, table_bet, table_hand):
        """Prompt user to make a decision about Hand Status within the rules of poker.
        
        If current_bet is an integer, tell the player the current bet is current_bet.
            Limits options to ["call","raise","fold"]
            If user selects "raise", call self.raise_bet()
            If user selects "call", subtracts (table.table_bet - self.player_bet) and resets self.player_bet to table.table_bet
            
        If current bet is 0, it must be check, so tell him that everyone else checked.
            Limits options to ["check","bet","fold"]
            If user selects "raise", lead him through raise control.
        
        Exhibits control and error handling for normal dimwittedness of users.
        
        Returns raise_amount - integer >= 0
        
        """
        
        print("Your hand is {0}. Your balance is {1}.".format(self.player_hand,self.balance))
        status = ""
        if table_bet > self.balance:
            print("The bet of {0} exceeds your balance of {1}. You have to fold.".format(table_bet,self.balance))
            self.hand_status = "fold"
        elif self.balance > 0 and table_bet == 0:
            while status.lower() not in ["check","raise","fold","quit","c","r","f","q"]:
                status = input("""Everybody else has checked. What would you like to do? You may enter:
                [B]est?: Ask to compute your best hand
                [C]heck: Bet 0 and pass to the next player
                [R]aise: Bet any amount between $0 and ${0}.
                [F]old:  Leave this round
                [Q]uit:  Quit the game.\n""".format(self.balance))
                
                if self.balance == 0:
                    print("You are all in. You check.")
                    self.player_bet = 0
                    self.hand_status = "check"
                else:
                    if status.lower() in ["b","best?"]:
                        self.players_best_hand(self.player_hand + table_hand)
                        print("Your best hand is {0}.\n".format(self.best_hand))
                        continue
                    elif status.lower() in ["c","check"]:
                        self.player_bet = 0
                        self.hand_status = "check"
                    elif status.lower() in ["r","raise"]:
                        self.player_bet = self.raise_bet(table_bet)
                        self.hand_status = "raise"
                    elif status.lower() in ["fold","f"]:
                        self.hand_status = "fold"
                    elif status.lower() in ["quit","q"]:
                        self.hand_status = "quit"
                    else:
                        print("Invalid Entry. Please select from the given options.")
        
        #otherwise, somebody has bet before you
        #if that's the case, choose from check, bet, or fold
        else:
            if self.balance == 0:
                print("You are all in. You call with 0.")
                self.player_bet = 0
                self.hand_status = "call"
            else:
                while status.lower() not in ["call","raise","fold","quit","c","r","f","q"]:
                    status = input("""The bet is {0}. What would you like to do? You may enter:
                    [B]est?: Ask to compute your best hand
                    [C]all:  Stay in the round by matching the bet on the table.
                    [R]aise: Bet any amount between ${0} and ${1}.
                    [F]old:  Leave this round.
                    [Q]uit:  Quit the game.\n""".format(table_bet, self.balance))

                    if status.lower() in ["b","best?"]:
                        self.players_best_hand(self.player_hand + table_hand)
                        print(self.best_hand)
                        continue
                    elif status.lower() in ["c","call"]:
                        self.player_bet = table_bet
                        self.balance -= self.player_bet
                        self.hand_status = "call"
                    elif status.lower() in ["r","raise"]:
                        self.player_bet = self.raise_bet(table_bet)
                        self.hand_status = "raise"
                        self.balance -= self.player_bet
                    elif status.lower() in ["fold","f"]:
                        self.hand_status = "fold"
                    elif status.lower() in ["quit","q"]:
                        self.hand_status = "quit"
                    else:
                        print("Invalid Entry. Please select from the given options.")
 
        return self.player_bet
    
    def raise_bet(self, table_bet):
        """When self.hand_status == 'raise', will prompt user to select:
        raise_amountt = float between amount from Table.table_bet to self.balance to add to Table.pot."""
        
        while True:
            raise_amount = input("""How much would you like to raise?
            You may raise anywhere between $0 and ${0}\n""".format(self.balance - table_bet))
        
            try:
                raise_amount = float(raise_amount)
                if 0 <= raise_amount <= (self.balance-table_bet):
                    raise_amount = int(raise_amount)
                    break
                else:
                    print("Invalid Entry. Please enter a value between {0} and {1}".format(table_bet,(self.balance-table_bet)))
            except ValueError:
                print("Invalid Entry. Please enter a number.")
        
        return raise_amount

class Game():
    def __init__(self):
        """Upon instantiation, request from user the number of players that will play the game.
        Store that number as self.num_players (pass to each hand)."""
        
        
        
        #generate the list of players to be fed to the game.
        self.player_list = [Bot("Bot{0}".format(i)) for i in range(int(round(rand.choice(list(range(2,22))))))]
        
        self.game_manager()
        
    def game_manager(self, quit=""):
        """When quit is not "quit" and user hasn't quit the game, instantiate a new hand"""
        #We will continue to start new hands until the user quits or runs out of money
        dealer = 0
        
        while True:
            #check to see if any player has run out of money
            for i, player in enumerate(self.player_list):
                #If the user ran out of money, quit the game
                if player.balance <= 0 and isinstance(player, Human):
                    print("You're out of money! Thank you for playing.")
                    quit = 'quit'
                    break
                #If a bot ran out of money, boot it from the game
                elif player.balance <= 0 and isinstance(player, Bot):
                    print("{0} has run out of money. {0} has left the game.".format(player.name))
                    del self.player_list[i]            
            
            #once you've done your checks, start a new hand by calling round_manager()
            quit = self.round_manager(dealer=dealer).lower()
            if quit in ["q","quit"]:
                break
            else:
                dealer += 1
                
    def __str__(self):
        return "The game has concluded. Thank you for playing!"
    
    def __repr__(self):
        return "The game has concluded. Thank you for playing!"
        
            
    def round_manager(self, quit="", dealer=0):
        """This is the actual operator of the game. Here, we will just move the hand along when each requirement is satisfied.
        We'll move the game along by calling the methods of class Hand in the appropriate order according to the rules
        
        Deal cards corresponding with the amount required for that round of betting:
        Round 1 = 3 Cards
        Round 2 = 1 Card
        Round 3 = 1 Card
        
        Once cards are dealt, call reset_player_status() and self.zero_bets()
        
        When Round 3 of betting concludes, call find_best_hand()"""

        while True:
            #Get a deck from the class Deck
            table_deck = Deck()
            
            current_hand = Round(table_deck.shuffle_deck(), self.player_list, dealer)

            #First, deal out everybody's hand
            for player in self.player_list:
                player.player_hand = table_deck.deal(1)
                
            #call set_dealer to set the big and small blinds
            current_hand.set_dealer()
            print("\n\nThe Big Blind is set at 100. Players will bet or call against this bet to begin.")
            
            #Start zeroth round of betting
            quit = current_hand.request_decision()
            if quit in ["q", "quit"]:
                return quit
            #Iterate through three rounds
            #The first round corresponds to the "flop", which is three cards. 
            #The second round is the "turn", which is one card
            #The third is the "river", which is one card
            for num_cards in [2,0,0]:
                current_hand.table_hand.extend(table_deck.deal(num_cards))
                print("The community cards are: \n {0}\n".format([x for x in current_hand.table_hand]))
                
                current_hand.zero_bets()
                current_hand.reset_player_status()
                quit = current_hand.request_decision()
                #If user enters 'quit' as their decision, we'll feed this up to quit the game. 
                if quit in ["q", "quit"]:
                    return quit
                
                #if everybody but one person folded in the hand, break the loop and find_best_hand()
                if [x.hand_status for x in self.player_list].count("fold") == len(self.player_list) - 1:
                    break
            
            
            current_hand.decide_winner()
            
            break
        return quit
        
class Round():
    def __init__(self, deck, player_list, dealer_index):
        """Contains the main attributes of the game, including:
        
        Player List - List of player instances, up to (52-5)/2 = 24
        Deck - instance of Deck, below
        Pot - Float, the money in the pot
        Table Bet - Float, the last bet made
        Dealer Index - the index of the dealer in self.player_list
        Betting Round - Integer between 0 and 4 indicating what stage of betting you're in (initial,flop,river,turn)
        Table Hand - List to be appended by deal_next_sequence, will go until length == 5.
        Winners - the list of winners with the best hand
        """
        
        self.player_list = player_list
        self.deck = deck
        self.player_status = {player.name:player.hand_status for player in player_list}
        self.pot = 0
        self.table_bet = 0
        self.table_hand = []
        self.dealer_index = dealer_index / len(player_list)
        self.winners = []
        
    def request_decision(self):
        """Iterate through player list, calling player.make_decision() for each.
                
        Continue to iterate until:
            - All players are set to 'check'
            - All players are set to 'call' or 'fold', but one is set to 'raise'
            - One player is set to 'raise' and the rest are set to 'fold'
        """
        
        n = 0
        while True:
            #The first time through betting, start betting left of the dealer
            #all subsequent times through the order will be assigned based on the last person who bet
            
            #set the order in which you will request the decision
            #if its the first time through, look to the left of the dealer = i+1
            if n == 0:
                i = "".join([str(player.dealer) for player in self.player_list]).find("1")
                self.player_list = self.player_list[i+1:] + self.player_list[:i+1]
                request_decision_order = self.player_list
            #otherwise, look to the right of the better = i+1
            else:
                i = "".join([str(player.raiser) for player in self.player_list]).find("1")
                request_decision_order = self.player_list[i+1:] + self.player_list[:i]
                        
            #this iterates through the player list from the left of the dealer/last bettor and requests decisions
            #if a human replies with "quit", then feed up the quit sequence
            #otherwise, if somebody raises, restart the loop
            for player in request_decision_order:
                bet = player.make_decision(self.table_bet, self.table_hand)
                #add the player's bet to the pot
                self.pot += bet
                if player.hand_status == 'raise':
                    #make sure that 'i' is assigned when you break to the top of the loop
                    self.table_bet += bet
                    for y in self.player_list:
                        y.raiser = 0
                    player.raiser = 1
                    n += 1
                    break
                if player.hand_status == 'fold':
                    self.player_list = [x for x in self.player_list if x.name != player.name]
                if player.hand_status == 'quit':
                    #if a human enters quit, stop the function and return
                    return 'quit'
            else:
                #if the for loop evaluates without breaking, then nobody raised again
                #if nobody raised again, then you've ended this round of betting
                return ""
    
    def zero_bets(self):
        """Reset all players' attribute Player.player_bet to 0 to begin new round of betting"""
        for player in self.player_list:
            player.player_bet = 0
            
        #Set table_bet to None to start the next round of betting
        self.table_bet = 0
    
    def reset_player_status(self):
        """Reset all players to empty string except for any player that folded"""
        for player in self.player_list:
            if player.hand_status != 'fold':
                player.hand_status = None
    
    def set_dealer(self):
        '''Each round needs a dealer. The index of the dealer will be passed with the player list.
        Set that player as the dealer by setting player.dealer = 1
        Then, set that players status to 'raise' and set his .player_bet to the ante
        Then, set the player to his right to a value of ante / 2. He status will start as None though.'''
        
        #clear dealer status of all players
        for player in self.player_list:
            player.dealer = 0
        
        dealer_int = int(self.dealer_index % len(self.player_list))
        #set dealer = 1 for the dealers index 
        #this is the mod of dealer index and length of player list since dealer index could be greater than that length
        self.player_list[dealer_int].dealer = 1
    
        for i, player in enumerate(self.player_list):
            if i == (dealer_int):
                #this person has the big blind, which counts as the raise of the first round
                self.hand_status = 'raise'
                #we're setting the big blind to 100
                player.balance -= 100
                player.player_bet = 100
                
                #we also need to set the table's bet to 100
                self.table_bet = 100
                
            elif i == 1 + (dealer_int):
                #this person is required to start with a small_blind = big_blind / 2
                #don't ask me why. its just the rules okay?
                player.balance -= 100
                self.player_bet = 50
                
    def decide_winner(self):
        """Instantiate Scorer with the hands of any Player with Status of 'call' after last round of betting.
        Extend all player's hands with Hand.table_hand
        If there's only one player left, he wins.
        
        Returns: Player whose Player.balance will be added to by Hand.pot, False
            -if decide_winner() finds multiple winners, divide pot by the length of that list and distribute"""
        
        best_hand_dict = {}
        
        for i, player in enumerate(self.player_list):
            player.players_best_hand(player.player_hand + self.table_hand)
            if i == 0:
                best_hand_dict[player] = player.best_hand
                
            elif player.best_hand > best_hand_dict[last_player]:
                best_hand_dict.clear()
                best_hand_dict[player] = player.best_hand
                
            elif player.best_hand == best_hand_dict[last_player]:
                best_hand_dict[player] = player.best_hand
                
            last_player = player
        
        print("The best hands of the players remaining at the table are:")
        print("\n".join([str(x.best_hand) for x in self.player_list]))
            
        #distribute pot to all winners in the winner dict
        self.winners = [x for x in best_hand_dict.keys()]
        divided_winnings = self.pot / len(self.winners)
        for player in self.winners:
            player.balance += divided_winnings
        if len(self.winners) == 1:
            print("The winner is {0} with hand of {1}".format(self.winners[0].name,self.winners[0].best_hand))
        else:
            print("The winners are {0} with a hand of {1}.".format("and ".join([x.name for x in self.winners]),self.winners[0].best_hand))
        print("{0} wins {1}.\n".format("and ".join([x.name for x in self.winners]),str(divided_winnings)))
            
class Hand():
    """This stores many attributes of a hand, including its "score". 
    
    Create a long train of if/elif that returns the rank of your hand if the hand meets those conditions.
    i.e. if five cards in the hand are of the same suit and those five cards are sequential face cards, return Royal Flush and break
    if not, move onto the straight flush, etc.

    Returns a list of length 6. The first element is the rank of your hand, the second through sixth are the ranks of the cards within that type of hand
    If you have a Full House of Aces over Kings, your list will be [7,14,14,14,13,13]
    If you have a Flush with a high card Ace with a low of 5, your list would be something like [6,14,10,9,6,5]"""
    
    def __init__(self,contents):
        self.contents = contents
        self.sorted_hand = sorted(contents)
        
        ###Test for the number of cards in a row from the beginning of the hand (used to find straights)
        self.consecutive_values = 0
        for i, card in enumerate(self.sorted_hand):
            #automatically you have one card in a row. good for you!
            if i == 0:
                self.consecutive_values += 1
            #if this card is exactly one rank greater than the last, its consecutive. you're on your way!
            elif card.rank == (last_card.rank + 1):
                self.consecutive_values += 1
            #but if its not one greater, no straight for you!
            else:
                break
            last_card = card
        
        ###Test for number of identical suits (to be used to find flushes)
        self.matched_suits = 0
        for i, card in enumerate(self.sorted_hand):
            #automatically you have one card in a suit. good for you!
            if i == 0:
                self.matched_suits += 1
            #if this card is the same suit as the last, it matches. you're on your way!
            elif card.suit == last_card.suit:
                self.matched_suits += 1
            #but if its not the same, no flush for you!
            else:
                break
            last_card = card
            
        ###Assign number of times card.rank reoccurs in the hand (to be used in finding pairs)
        self.pair_dict = {}
        for card in self.sorted_hand:
            if card.rank not in self.pair_dict.keys():
                self.pair_dict[card.rank] = 1
            else:
                self.pair_dict[card.rank] += 1
        
        #Assign the highest card in the hand (used for finding high card)
        self.highest_card = self.sorted_hand[-1]
        
        ###Now, assign the hand's "score", described below
        #Royal Flush
        if self.consecutive_values == 5 and self.matched_suits == 5 and self.highest_card.rank == 14:
            self.hand_score = [10] + [x.rank for x in self.sorted_hand]
        #Straight Flush
        elif self.consecutive_values == 5 and self.matched_suits == 5 and self.highest_card.rank < 14:
            self.hand_score = [9] + [x.rank for x in self.sorted_hand]
        #Four of a Kind
        elif max([x for x in self.pair_dict.values()]) == 4:
            self.hand_score = [8] + [x.rank for x in sorted(test_hand)]
        #Full House
        elif max([x for x in self.pair_dict.values()]) == 3 and min([x for x in self.pair_dict.values()]) == 2:
            self.hand_score = [7] + [x.rank for x in self.sorted_hand]
        #Flush
        elif self.consecutive_values < 5 and self.matched_suits == 5:
            self.hand_score = [6] + [x.rank for x in self.sorted_hand]
        #Straight
        elif self.consecutive_values == 5 and self.matched_suits < 5:
            self.hand_score = [5] + [x.rank for x in self.sorted_hand]
        #Three of a Kind
        elif max([x for x in self.pair_dict.values()]) == 3 and min([x for x in self.pair_dict.values()]) < 2:
            self.hand_score = [4] + [x.rank for x in self.sorted_hand]
        #Two Pair
        elif max([x for x in self.pair_dict.values()]) == 2 and [x for x in self.pair_dict.values()].count(2) == 2:
            self.hand_score = [3] + [x.rank for x in self.sorted_hand]
        #Pair
        elif max([x for x in self.pair_dict.values()]) == 2 and [x for x in self.pair_dict.values()].count(2) == 1:
            self.hand_score = [2] + [x.rank for x in self.sorted_hand]
        #High Card
        else:
            self.hand_score = [1] + [x.rank for x in self.sorted_hand]
                
    def __gt__(self,other):
        return self.hand_score > other.hand_score

    def __lt__(self,other):
        return self.hand_score < other.hand_score

    def __eq__(self,other):
        return self.hand_score == other.hand_score

    def __str__(self):
        return " ".join([str(x) for x in self.contents])

    def __repr__(self):
        return " ".join([str(x) for x in self.contents])
        
class PlayingCard():
    def __init__(self, suit, rank):
        """Initialize PlayingCard with attributes Suit and Rank."""
        self.rank = rank
        self.suit = suit
        
    def __eq__(self,other):
        if self.rank == other.rank:
            return True
        else:
            return False
        
    def __gt__(self, other):
        if self.rank > other.rank:
            return True
        else:
            return False
    
    def __lt__(self,other):
        if self.rank < self.rank:
            return True
        else:
            return False
        
    def __str__(self):
        if self.rank == 11:
            return "J{}".format(self.suit)
        elif self.rank == 12:
            return "Q{}".format(self.suit)
        elif self.rank == 13:
            return "K{}".format(self.suit)
        elif self.rank == 14:
            return "A{}".format(self.suit)
        else:
            return "{0}{1}".format(self.rank,self.suit)
    
    def __repr__(self):
        if self.rank == 11:
            return "J{}".format(self.suit)
        elif self.rank == 12:
            return "Q{}".format(self.suit)
        elif self.rank == 13:
            return "K{}".format(self.suit)
        elif self.rank == 14:
            return "A{}".format(self.suit)
        else:
            return "{0}{1}".format(self.rank,self.suit)
        
    def __hash__(self):
        if self.rank == 11:
            return hash("J{}".format(self.suit))
        elif self.rank == 12:
            return hash("Q{}".format(self.suit))
        elif self.rank == 13:
            return hash("K{}".format(self.suit))
        elif self.rank == 14:
            return hash("A{}".format(self.suit))
        else:
            return hash("{0}{1}".format(self.rank,self.suit))
        
class Deck():
    
    card_values = list(range(2,15))
    
    def __init__(self,suit="all"):
        """Initialize deck with 52 unique of PlayingCards."""
        if suit == "all":
            self.cards = [PlayingCard(x,y) for x in ["D","H","S","C"] for y in Deck.card_values]
        elif suit.upper() in ["D","H","S","C"] and len(suit) == 1:
            self.cards = [PlayingCard(x,suit.upper()) for x in Deck.card_values]
        else:
            print("Invalid entry for suit. Deck not created.")
            return None
        
        self.cards = self.shuffle_deck()
    
    def shuffle_deck(self):
        """Randomize self.cards"""
        #do shuffle process 50 times
        for i in range(50):
            #get a random number to split the deck on
            j = rand.randint(0,len(self.cards)+1)
            #select all even indices to come first, all odd to come second
            self.cards = [x for i, x in enumerate(self.cards) if i%2==0] + [x for i, x in enumerate(self.cards) if i%2==1]
            #and then bring the back [j:] to the front
            self.cards = self.cards[j:] + self.cards[:j]
        return self.cards
    
    def deal(self, card_count=0):
        """Pops card_count cards and returns them to user. 
        Returns None with message when len(cards)==0 or len(cards) > card_count"""
        
        #check length of self.cards and return card_count cards when you can
        if len(self.cards)==0:
            print("Your deck is out of cards.")
            return []
        
        elif len(self.cards) >= card_count:
            return [self.cards.pop(x) for x in range(card_count,-1,-1)]
        
        else:
            print("Your deck has fewer than {0} cards.".format(card_count))
            return []


In [3]:
my_game = Game()
my_game



The Big Blind is set at 100. Players will bet or call against this bet to begin.
Bot1 folds.
Bot2 calls.
Bot3 folds.
Bot4 calls.
Bot5 raises 48.
The last bet on the table is 148.
Bot6 calls.
Bot7 calls.
Bot8 raises 60.
The last bet on the table is 208.
Bot9 calls.
Bot10 folds.
Bot11 calls.
Bot12 calls.
Bot13 raises 34.
The last bet on the table is 242.
Bot14 raises 99.
The last bet on the table is 341.
Bot15 calls.
Bot0 folds.
Bot2 calls.
Bot4 folds.
Bot5 folds.
Bot6 calls.
Bot7 folds.
Bot8 raises 1.
The last bet on the table is 342.
Bot9 calls.
Bot11 calls.
Bot12 calls.
Bot13 calls.
Bot14 folds.
Bot15 raises 30.
The last bet on the table is 372.
Bot2 raises 6.
The last bet on the table is 378.
Bot6 folds.
Bot8 calls.
Bot9 folds.
Bot11 folds.
Bot12 calls.
Bot13 folds.
Bot15 raises 11.
The last bet on the table is 389.
Bot2 folds.
Bot8 folds.
Bot12 folds.
The community cards are: 
 [7S, JD, 5D]

Bot15 checks.
The best hands of the players remaining at the table are:
10S 9C 7S JD 5D
Th

KeyError: Bot4